# Building an LLM Application with Langchain

This notebook is mostly code.  For more context, see this [blog post](https://blog.agilephd.com/posts/llm_langchain/)

In [19]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_openai import OpenAI

load_dotenv()

True

In [20]:
loader = PyPDFLoader("pm-11.pdf")
data = loader.load()
# print(data[0])

In [21]:
policy_text = ""
for doc in data:
    if isinstance(doc, dict) and 'text' in doc:
        policy_text += doc['text']
    elif isinstance(doc, str):
        policy_text += doc
    else:
        policy_text += repr(doc)

In [22]:
ct_splitter = CharacterTextSplitter(separator='.', chunk_size=1000, chunk_overlap=200)
docs = ct_splitter.split_text(policy_text)
# print(docs)

In [23]:
api_key = os.getenv("OPENAI_API_KEY")
embedding_function = OpenAIEmbeddings(openai_api_key=api_key)

# vectordb = Chroma(persist_directory='LSU', embedding_function=embedding_function)

# docstorage = Chroma.from_texts(docs, embedding_function)
docstorage = FAISS.from_texts(docs, embedding_function)

In [24]:
llm=OpenAI(model_name="gpt-3.5-turbo-instruct", openai_api_key=api_key)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docstorage.as_retriever())                             

In [25]:
question = "Can I get a blanket approval for work outside of LSU?"
response = qa.invoke(question)

In [26]:
print(response['result'])

 No, according to section IV of the document, blanket approvals for outside employment will not be granted.
